In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# %%
import torch
import string

In [ ]:
!pip install sentencepiece

In [ ]:
!pip install transformers

##BERT

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
cd drive/My Drive/Colab Notebooks/experiments

/content/drive/My Drive/Colab Notebooks/experiments


In [ ]:
# Use a subset for quick experiments
#data = data[:10000]

from sklearn.model_selection import train_test_split as tts
import pandas as pd
data = pd.read_csv("data/moh-x.csv")

# Split to train, val and test
text, test_df = tts(data[["sentence", "arg1", "verb", "label"]], random_state=42, test_size=0.1)
text, val = tts(text, random_state=42, test_size=test_df.shape[0])

In [ ]:
text = text["sentence"]

In [ ]:
#text.to_csv(r'mohx.txt', header=None, index=None, sep=' ', mode='a')

In [ ]:
with open('data/mohx.txt', 'r') as fp:
    text = fp.read().split('\n')

In [ ]:
text[:5]

['" I ca n\'t buy this story ."',
 '" European children learn the breast stroke ; they often do n\'t know how to crawl ."',
 '" He shed his image as a pushy boss ."',
 '" She devoured his novels ."',
 '" Age and experience mellowed him over the years ."']

In [ ]:
inputs = tokenizer(text, return_tensors='pt', max_length=21, truncation=True, padding='max_length')

In [ ]:
inputs

{'input_ids': tensor([[  101,  1000,  1045,  ...,     0,     0,     0],
        [  101,  1000,  2647,  ..., 13529,  1012,   102],
        [  101,  1000,  2002,  ...,     0,     0,     0],
        ...,
        [  101,  1000, 13862,  ...,     0,     0,     0],
        [  101,  1000,  2016,  ...,     0,     0,     0],
        [  101,   102,     0,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 0,  ..., 0, 0, 0]])}

In [ ]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [ ]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [ ]:
# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
           (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [ ]:
mask_arr

tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False,  True, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])

In [ ]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

In [ ]:
selection[:5]

[[6, 9], [12, 13], [7, 11], [], [5, 7, 11]]

In [ ]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [ ]:
inputs.input_ids

tensor([[  101,  1000,  1045,  ...,     0,     0,     0],
        [  101,  1000,  2647,  ..., 13529,  1012,   102],
        [  101,  1000,  2002,  ...,     0,     0,     0],
        ...,
        [  101,   103, 13862,  ...,     0,     0,     0],
        [  101,  1000,  2016,  ...,     0,     0,     0],
        [  101,   102,     0,  ...,     0,     0,     0]])

In [ ]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
dataset = MeditationsDataset(inputs)

In [ ]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [ ]:
#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
#model.to(device)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [ ]:
from transformers import AdamW

# activate training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=5e-5)

In [ ]:
from tqdm import tqdm  # for our progress bar

epochs = 3

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())


  0%|          | 0/33 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """

Epoch 0: 100%|██████████| 33/33 [01:01<00:00,  1.85s/it, loss=0.529]

Epoch 1: 100%|██████████| 33/33 [00:59<00:00,  1.80s/it, loss=0.128]

Epoch 2: 100%|██████████| 33/33 [00:59<00:00,  1.80s/it, loss=0.131]


##Save Model

In [ ]:
import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = 'stockholm/mlm/mohx_bert_finetuned/'
# output_dir = './content/xlm-roberta_model_save/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Good practice: save your training arguments together with the trained model
# torch.save(args, os.path.join(output_dir, 'training_args.bin'))

Saving model to stockholm/mlm/mohx_bert_finetuned/


('stockholm/mlm/mohx_bert_finetuned/tokenizer_config.json',
 'stockholm/mlm/mohx_bert_finetuned/special_tokens_map.json',
 'stockholm/mlm/mohx_bert_finetuned/vocab.txt',
 'stockholm/mlm/mohx_bert_finetuned/added_tokens.json')

##Import Model

In [ ]:
!pip install transformers

from transformers import XLMRobertaForSequenceClassification

output_dir = 'stockholm/mlm/mohx_bert_finetuned/'

print(output_dir)

stockholm/mlm/mohx_bert_finetuned/


In [ ]:
from transformers import BertTokenizerFast, BertConfig, BertForSequenceClassification, AutoModel
import torch
# Load the BERT tokenizer.
print('Loading Bert Base Uncased Tokenizer...')
bert_tokenizer = BertTokenizerFast.from_pretrained(output_dir)
model = torch.load('stockholm/mlm/mohx_bert_finetuned/pytorch_model.bin')

Loading Bert Base Uncased Tokenizer...


##XLM

In [ ]:
from transformers import XLMRobertaTokenizer, XLMRobertaForMaskedLM
xlmroberta_tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
xlmroberta_model = XLMRobertaForMaskedLM.from_pretrained('xlm-roberta-base')

In [ ]:
inputs = xlmroberta_tokenizer(text, return_tensors='pt', max_length=21, truncation=True, padding='max_length')

In [ ]:
inputs

{'input_ids': tensor([[     0,     44,     87,  ...,      1,      1,      1],
        [     0,     44,  28811,  ..., 235879,      6,      2],
        [     0,     44,   1529,  ...,      1,      1,      1],
        ...,
        [     0,     44,  14838,  ...,      1,      1,      1],
        [     0,     44,   4687,  ...,      1,      1,      1],
        [     0,      2,      1,  ...,      1,      1,      1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 0,  ..., 0, 0, 0]])}

In [ ]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [ ]:
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [ ]:
# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
           (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [ ]:
mask_arr

tensor([[False,  True, False,  ...,  True, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False,  True, False,  ..., False, False, False],
        [False, False, False,  ...,  True,  True,  True],
        [False, False, False,  ..., False, False, False]])

In [ ]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

In [ ]:
selection[:5]

[[1, 5, 6, 15, 17, 18], [7, 12], [3, 17], [4, 11, 14], [6, 11]]

In [ ]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [ ]:
inputs.input_ids

tensor([[     0,    103,     87,  ...,    103,      1,      1],
        [     0,     44,  28811,  ..., 235879,      6,      2],
        [     0,     44,   1529,  ...,      1,      1,      1],
        ...,
        [     0,    103,  14838,  ...,      1,      1,      1],
        [     0,     44,   4687,  ...,    103,    103,    103],
        [     0,      2,      1,  ...,      1,      1,      1]])

In [ ]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
dataset = MeditationsDataset(inputs)

In [ ]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [ ]:
#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
#xlmroberta_model.to(device)

XLMRobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Laye

In [ ]:
from transformers import AdamW

# activate training mode
xlmroberta_model.train()
# initialize optimizer
optim = AdamW(xlmroberta_model.parameters(), lr=5e-5)

In [ ]:
from tqdm import tqdm  # for our progress bar

epochs = 3

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        # process
        outputs = xlmroberta_model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())


  0%|          | 0/33 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """

Epoch 0: 100%|██████████| 33/33 [03:04<00:00,  5.59s/it, loss=1.61]

Epoch 1: 100%|██████████| 33/33 [02:37<00:00,  4.78s/it, loss=0.734]

Epoch 2: 100%|██████████| 33/33 [02:37<00:00,  4.78s/it, loss=0.251]


In [ ]:
bert_model = model.eval()

In [ ]:
xlmroberta_model = xlmroberta_model.eval()

##Masked Language Modeling Pipeline

In [ ]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
from transformers import XLNetTokenizer, XLNetLMHeadModel
xlnet_tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
xlnet_model = XLNetLMHeadModel.from_pretrained('xlnet-base-cased').eval()

In [ ]:
from transformers import XLMRobertaTokenizer, XLMRobertaForMaskedLM
xlmroberta_tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large').eval()

In [ ]:
from transformers import ElectraTokenizer, ElectraForMaskedLM
electra_tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-generator')
electra_model = ElectraForMaskedLM.from_pretrained('google/electra-small-generator').eval()

In [ ]:
from transformers import RobertaTokenizer, RobertaForMaskedLM
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta_model = RobertaForMaskedLM.from_pretrained('roberta-base').eval()

In [ ]:
top_k = 10

In [ ]:
def decode(tokenizer, pred_idx, top_clean):
    ignore_tokens = string.punctuation + '[PAD]'
    tokens = []
    for w in pred_idx:
        token = ''.join(tokenizer.decode(w).split())
        if token not in ignore_tokens:
            tokens.append(token.replace('##', ''))
    return '\n'.join(tokens[:top_clean])

In [ ]:
def encode(tokenizer, text_sentence, add_special_tokens=True):
    text_sentence = text_sentence.replace('<mask>', tokenizer.mask_token)
    # if <mask> is the last token, append a "." so that models dont predict punctuation.
    if tokenizer.mask_token == text_sentence.split()[-1]:
        text_sentence += ' .'

    input_ids = torch.tensor([tokenizer.encode(text_sentence, add_special_tokens=add_special_tokens)])
    mask_idx = torch.where(input_ids == tokenizer.mask_token_id)[1].tolist()[0]
    return input_ids, mask_idx

In [ ]:
def get_all_predictions(text_sentence, top_clean=5):
    # ========================= BERT =================================
    print(text_sentence)
    input_ids, mask_idx = encode(bert_tokenizer, text_sentence)
    with torch.no_grad():
        predict = model(input_ids)[0]
    bert = decode(bert_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= XLNET LARGE =================================
    input_ids, mask_idx = encode(xlnet_tokenizer, text_sentence, False)
    perm_mask = torch.zeros((1, input_ids.shape[1], input_ids.shape[1]), dtype=torch.float)
    perm_mask[:, :, mask_idx] = 1.0  # Previous tokens don't see last token
    target_mapping = torch.zeros((1, 1, input_ids.shape[1]), dtype=torch.float)  # Shape [1, 1, seq_length] => let's predict one token
    target_mapping[0, 0, mask_idx] = 1.0  # Our first (and only) prediction will be the last token of the sequence (the masked token)

    with torch.no_grad():
        predict = xlnet_model(input_ids, perm_mask=perm_mask, target_mapping=target_mapping)[0]
    xlnet = decode(xlnet_tokenizer, predict[0, 0, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= XLM ROBERTA BASE =================================
    input_ids, mask_idx = encode(xlmroberta_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = xlmroberta_model(input_ids)[0]
    xlm = decode(xlmroberta_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= BART =================================
    input_ids, mask_idx = encode(bart_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = bart_model(input_ids)[0]
    bart = decode(bart_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= ELECTRA =================================
    input_ids, mask_idx = encode(electra_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = electra_model(input_ids)[0]
    electra = decode(electra_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= ROBERTA =================================
    input_ids, mask_idx = encode(roberta_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = roberta_model(input_ids)[0]
    roberta = decode(roberta_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    return {'bert': bert,
            'xlnet': xlnet,
            'xlm': xlm,
            'bart': bart,
            'electra': electra,
            'roberta': roberta}

##MOH-X

In [ ]:
text_sentence = 'The stars <mask> towards each other'

In [ ]:
get_all_predictions(text_sentence)

The stars <mask> towards each other


{'bart': 'are\nturn\nwere\nand\nhave',
 'bert': 'pointed\nstreaked\ndrifted\nsped\nshone',
 'electra': 'turn\nlook\nmove\nwave\nlean',
 'roberta': 'look\nmove\nturn\nwalk\nturned',
 'xlm': 'turn\nspin\ndrift\nrun\nspread',
 'xlnet': 'towards\nturn\nstart\nin\nare'}

In [ ]:
text_sentence = 'The <mask> gravitate towards each other'

In [ ]:
get_all_predictions(text_sentence)

The <mask> gravitate towards each other


{'bart': 'two\nstars\nthree\nway\nworld',
 'bert': 'two\npair\npeople\nmen\nbrothers',
 'electra': 'two\nthree\npeople\nmen\nchildren',
 'roberta': 'characters\nstudents\ntwo\nplayers\nthree',
 'xlm': 'animals\nwomen\nmen\ncharacters\nfish',
 'xlnet': 'a\nI\nthe\nand\nThe'}

In [ ]:
text_sentence = 'A hot <mask> will revive me'

In [ ]:
get_all_predictions(text_sentence)

A hot <mask> will revive me


{'bart': 'shower\ncup\nbath\ntodd\nchocolate',
 'bert': 'bath\nshower\nsun\nmeal\ndrink',
 'electra': 'summer\nspring\nsun\nday\nmoment',
 'roberta': 'bath\nshower\ndrink\nday\niron',
 'xlm': 'bath\nwater\nsauce\nshower\nsummer',
 'xlnet': 'will\nwhich\nhot\nthat\nstar'}

In [ ]:
text_sentence = 'A hot soup will <mask> me'

In [ ]:
get_all_predictions(text_sentence)

A hot soup will <mask> me


{'bart': 'do\nalways\ncure\nnever\nnot',
 'bert': 'warm\nkill\ncure\ncool\nsoothe',
 'electra': 'give\nhelp\nsurprise\nsave\ntell',
 'roberta': 'calm\ncure\nrevive\nhelp\ncomfort',
 'xlm': 'kill\nplease\nhelp\nfeed\nmake',
 'xlnet': 'me\ndo\nit\nto\nbe'}

In [ ]:
text_sentence = 'They <mask> their way to the top of the mountain .'

In [ ]:
get_all_predictions(text_sentence)

They <mask> their way to the top of the mountain .


{'bart': 'then\nwere\nclimbed\nalso\nhad',
 'bert': 'made\nclawed\npushed\nworked\nfought',
 'electra': 'made\nmake\nfound\nfind\nclawed',
 'roberta': 'made\nmake\nfound\nclimbed\nclimb',
 'xlm': 'made\nfound\nmake\ndug\nworked',
 'xlnet': 'their\nTheir\nYour\nmade\nHis'}

In [ ]:
text_sentence = 'They clawed their <mask> to the top of the mountain .'

In [ ]:
get_all_predictions(text_sentence)

They clawed their <mask> to the top of the mountain .


{'bart': 'way\nown\nfeet\nhands\nfingers',
 'bert': 'way\nnails\nlegs\nhands\nheels',
 'electra': 'way\nways\nhands\npath\nclaws',
 'roberta': 'way\nfeet\nfingers\nways\nhands',
 'xlm': 'arms\nhands\nhead\nfeet\nway',
 'xlnet': 'way\nup\nupward\nback\nit'}

In [ ]:
text_sentence = 'He revived this <mask> of opera .'

In [ ]:
get_all_predictions(text_sentence)

He revived this <mask> of opera .


{'bart': 'work\nstyle\ntheme\ntradition\nopera',
 'bert': 'form\ntype\nwork\nstyle\ngenre',
 'electra': 'style\nform\nkind\ntype\ntradition',
 'roberta': 'form\nkind\ngenre\nstyle\ntype',
 'xlm': 'piece\nword\nkind\nspace\nstyle',
 'xlnet': 'this\nrevival\ntradition\nform\ntype'}

In [ ]:
text_sentence = 'He <mask> this style of opera .'

In [ ]:
get_all_predictions(text_sentence)

He <mask> this style of opera .


{'bart': 'is\nwrote\nalso\nwas\nhas',
 'bert': 'loved\nfavored\ndespised\npioneered\nhated',
 'electra': 'likes\nloves\nloved\nliked\nchose',
 'roberta': 'liked\nprefers\npreferred\ncreated\nloved',
 'xlm': 'ran\nwrote\nwon\nenjoyed\nmade',
 'xlnet': 'He\nThis\nShe\nhe\nIt'}

In [ ]:
text_sentence = 'They clawed their <mask> to the top of the mountain .'

In [ ]:
get_all_predictions(text_sentence)

They clawed their <mask> to the top of the mountain .


{'bart': 'way\nown\nfeet\nhands\nfingers',
 'bert': 'way\nnails\nlegs\nhands\nheels',
 'electra': 'way\nways\nhands\npath\nclaws',
 'roberta': 'way\nfeet\nfingers\nways\nhands',
 'xlm': 'arms\nhands\nhead\nfeet\nway',
 'xlnet': 'way\nup\nupward\nback\nit'}

In [ ]:
text_sentence = 'They <mask> their way to the top of the mountain .'

In [ ]:
get_all_predictions(text_sentence)

They <mask> their way to the top of the mountain .


{'bart': 'then\nwere\nclimbed\nalso\nhad',
 'bert': 'made\nclawed\npushed\nworked\nfought',
 'electra': 'made\nmake\nfound\nfind\nclawed',
 'roberta': 'made\nmake\nfound\nclimbed\nclimb',
 'xlm': 'made\nfound\nmake\ndug\nworked',
 'xlnet': 'their\nTheir\nYour\nmade\nHis'}

In [ ]:
text_sentence = 'We rotate the <mask> so as to maximize the use of the soil .'

In [ ]:
get_all_predictions(text_sentence)

We rotate the <mask> so as to maximize the use of the soil .


{'bart': 'soil\nplant\nroot\nwheel\nplants',
 'bert': 'soil\nearth\nsoils\nground\nland',
 'electra': 'water\nplants\nplates\nsystem\nbases',
 'roberta': 'soil\ncompost\ncrop\nplant\nplants',
 'xlm': 'water\nriver\nwind\nground\nearth',
 'xlnet': 'so\nin\nthe\nabout\nto'}

In [ ]:
text_sentence = 'We <mask> the crops so as to maximize the use of the soil .'

In [ ]:
get_all_predictions(text_sentence)

We <mask> the crops so as to maximize the use of the soil .


{'bart': 'can\nshould\nneed\nalso\nhave',
 'bert': 'grow\nplant\nplanted\ngrew\nwatered',
 'electra': 'grow\nplant\nprepare\nbuild\nharvest',
 'roberta': 'rotate\nplanted\ngrow\nseeded\ncombine',
 'xlm': 'spin\nsplit\ncut\nwater\nspread',
 'xlnet': 'the\nw\nWe\nwe\nThe'}

In [ ]:
text_sentence = "The <mask> don't harmonize ."

In [ ]:
get_all_predictions(text_sentence)

The <mask> don't harmonize .


{'bart': 'two\nwords\nrules\nnumbers\nlyrics',
 'bert': 'pieces\nsongs\nwords\nvoices\nmusicians',
 'electra': 'two\nelements\nwords\nthings\nlines',
 'roberta': 'two\nlanguages\nrules\nwords\nthree',
 'xlm': 'weather\nwater\narms\ningredients\nparties',
 'xlnet': 'They\nthey\nThe\nthe\ne'}

In [ ]:
text_sentence = "The colors don't <mask> ."

In [ ]:
get_all_predictions(text_sentence)

The colors don't <mask> .


{'bart': 'match\nchange\nlook\nreally\nshow',
 'bert': 'match\nchange\nmatter\nfade\nmix',
 'electra': 'change\nmatch\nwork\nmatter\nshow',
 'roberta': 'match\nchange\nmix\nmatter\nwork',
 'xlm': 'change\nmix\nmatch\nblend\ndiffer',
 'xlnet': 't\nnot\nand\ndo\nreally'}

In [ ]:
text_sentence = "The <mask> is being clogged by these operations ."

In [ ]:
get_all_predictions(text_sentence)

The <mask> is being clogged by these operations .


{'bart': 'entire\nsystem\nwhole\nwater\nmain',
 'bert': 'river\neconomy\nwater\ncountry\nindustry',
 'electra': 'field\nsystem\nnetwork\nsite\ninternet',
 'roberta': 'system\npipeline\npipe\nriver\nnetwork',
 'xlm': 'water\nriver\nearth\nocean\nmarket',
 'xlnet': 'is\nIt\nm\nwas\nAm'}

In [ ]:
text_sentence = "The market is being <mask> by these operations ."

In [ ]:
get_all_predictions(text_sentence)

The market is being <mask> by these operations .


{'bart': 'served\nfurther\ndominated\ndriven\nsupported',
 'bert': 'dominated\nflooded\naffected\nserved\npushed',
 'electra': 'fed\ndriven\naffected\ndominated\ncontrolled',
 'roberta': 'served\ndominated\npenetrated\ndriven\ndisrupted',
 'xlm': 'run\naffected\nraised\nreached\nspread',
 'xlnet': 'being\ngoing\ning\ntaking\nwas'}

In [ ]:
text_sentence = "The <mask> was swelling with importance when she spoke of her son ."

In [ ]:
get_all_predictions(text_sentence)

The <mask> was swelling with importance when she spoke of her son .


{'bart': 'mother\nwoman\nyoung\nformer\nactress',
 'bert': 'room\ndowager\nwoman\nair\nduchess',
 'electra': 'woman\nbody\nfamily\nheart\nworld',
 'roberta': 'room\nvoice\nair\nheart\ncongregation',
 'xlm': 'mother\nwoman\nwife\nfather\nowner',
 'xlnet': 'was\nIt\nI\nBody\nCenter'}

In [ ]:
text_sentence = "The mother was <mask> with importance when she spoke of her son ."

In [ ]:
get_all_predictions(text_sentence)

The mother was <mask> with importance when she spoke of her son .


{'bart': 'visibly\nalways\nalso\nnot\nsaid',
 'bert': 'filled\nheavy\nflushed\nfilling\nvibrating',
 'electra': 'filled\ntreated\nmet\npresented\ncharged',
 'roberta': 'speaking\nfilled\nglowing\nstruck\noverwhelmed',
 'xlm': 'raised\ntreated\nserved\nimpressed\nassociated',
 'xlnet': 'was\nof\nvery\nin\ngreat'}

In [ ]:
text_sentence = "Their <mask> inclines us to believe them ."

In [ ]:
get_all_predictions(text_sentence)

Their <mask> inclines us to believe them .


{'bart': 'own\nlack\nhistory\nsilence\narrogance',
 'bert': 'attitude\ntestimony\ncourage\nsilence\nfaith',
 'electra': 'belief\npresence\nfaith\nvoice\ncourage',
 'roberta': 'logic\nsincerity\nfaith\nsilence\ntruth',
 'xlm': 'faith\npower\nbelief\nstrength\nenergy',
 'xlnet': 'Their\nFaith\nGod\ntheir\nThey'}

In [ ]:
text_sentence = "Their language <mask> us to believe them ."

In [ ]:
get_all_predictions(text_sentence)

Their language <mask> us to believe them .


{'bart': 'is\nand\nmay\nhas\ntells',
 'bert': 'urged\nled\nforced\nleads\nencouraged',
 'electra': 'allows\ntells\nwants\nallowed\nrequires',
 'roberta': 'begs\nwants\ntells\nforces\ninvites',
 'xlm': 'told\nallows\nallowed\nhelped\ntaught',
 'xlnet': 'language\nis\nlanguages\nyou\nus'}

In [ ]:
text_sentence = "capture the <mask> of Spring   ."

In [ ]:
get_all_predictions(text_sentence)

capture the <mask> of Spring   .


{'bart': 'beauty\nspirit\nmoment\nmagic\nessence',
 'bert': 'scent\nheat\nbloom\nlight\nwater',
 'electra': 'image\nlight\nscent\nblood\nshadow',
 'roberta': 'spirit\nbeauty\npromise\nRite\nSpirit',
 'xlm': 'air\nwater\nheat\njoy\nflavor',
 'xlnet': 'of\nthe\nspirit\nday\nculture'}

In [ ]:
text_sentence = "<mask> the essence of Spring   ."

In [ ]:
get_all_predictions(text_sentence)

<mask> the essence of Spring   .


{'bart': 'Spring\n5\n4\n3\nS',
 'bert': 'taste\ndrain\nfollowing\ncontaining\nand',
 'electra': 'in\nand\nconsider\nsee\nbring',
 'roberta': 'Understanding\nand\nAnd\n...\nFinding',
 'xlm': 'repair\nEnjoy\nexpress\nwelcome\ndiscuss',
 'xlnet': 'the\nelement\nmain\nessence\nspirit'}

In [ ]:
text_sentence = "He showered her with <mask> ."

In [ ]:
get_all_predictions(text_sentence)

He showered her with <mask> .


{'bart': 'kisses\ngifts\naffection\na\nlove',
 'bert': 'kisses\nsoap\npaint\nwater\ngifts',
 'electra': 'water\nkisses\nflowers\ntea\ntears',
 'roberta': 'gifts\nflowers\nkisses\ncompliments\npraise',
 'xlm': 'water\nmilk\nbutter\noil\nsalt',
 'xlnet': 'a\nwith\nby\nand\nthis'}

In [ ]:
text_sentence = "He <mask> her with presents ."

In [ ]:
get_all_predictions(text_sentence)

He <mask> her with presents .


{'bart': 'also\nthen\nwas\nwould\nalways',
 'bert': 'showered\npresented\ndecorated\npresents\nprovided',
 'electra': 'provided\npresented\nfilled\nshowered\noffered',
 'roberta': 'surprised\npresented\npresents\nspoiled\ngifted',
 'xlm': 'surprised\nsed\nprovides\nfed\nbore',
 'xlnet': 'She\nHe\nHer\nWe\nIt'}

In [ ]:
text_sentence = "Am I supposed to swallow that <mask> ?"

In [ ]:
get_all_predictions(text_sentence)

Am I supposed to swallow that <mask> ?


{'bart': 'pill\nwhole\nshit\nand\none',
 'bert': 'pill\ninformation\npunch\nserum\nfear',
 'electra': 'pill\nshit\nthing\nnow\none',
 'roberta': 'pill\nshit\ncrap\none\ncake',
 'xlm': 'butter\nmilk\ncock\nworm\nwater',
 'xlnet': 'that\nthis\na\nwhich\nthen'}

In [ ]:
text_sentence = "Am I supposed to <mask> that story ?"

In [ ]:
get_all_predictions(text_sentence)

Am I supposed to <mask> that story ?


{'bart': 'believe\nbe\nfeel\nbuy\nlike',
 'bert': 'believe\ntell\nwrite\nshare\nfollow',
 'electra': 'tell\nwrite\nhave\nread\nknow',
 'roberta': 'tell\nbelieve\nwrite\nknow\nremember',
 'xlm': 'tell\nwrite\nread\nrun\ncover',
 'xlnet': 'write\ntell\nthat\nto\nbe'}

In [ ]:
text_sentence = "The <mask> glared down on us ."

In [ ]:
get_all_predictions(text_sentence)

The <mask> glared down on us .


{'bart': 'sun\ndark\nold\nblack\ntwo',
 'bert': 'sun\nlight\nteacher\njudge\nmoon',
 'electra': 'sun\nstars\ncamera\nclouds\nmen',
 'roberta': 'sun\ngods\nworld\nsky\nGods',
 'xlm': 'water\nsun\nwind\nstar\nsnow',
 'xlnet': 'The\nIt\nThey\nTo\nI'}

In [ ]:
text_sentence = "The sun <mask> down on us ."

In [ ]:
get_all_predictions(text_sentence)

The sun <mask> down on us .


{'bart': 'is\nwill\nwas\nhas\ncomes',
 'bert': 'beat\nshone\nglared\nblazed\nbore',
 'electra': 'came\nwas\nwent\nshone\nis',
 'roberta': 'beat\ncame\nbeats\nwent\nshone',
 'xlm': 'went\nturned\nrose\nlet\ngoes',
 'xlnet': 'down\nsun\nup\nback\nbody'}

In [ ]:
text_sentence = "<mask> erupted in the country ."

In [ ]:
get_all_predictions(text_sentence)

<mask> erupted in the country .


{'bart': 'An\nThe\nAnother\nMore\nT',
 'bert': 'violence\nprotests\nriots\nchaos\nfighting',
 'electra': 'riots\nviolence\ngunfire\nexplosions\nfire',
 'roberta': 'Fighting\nWar\nviolence\nViolence\nPolitics',
 'xlm': 'Spring\nFire\nspring\nworm\nPain',
 'xlnet': 'erupted\nin\nexploded\nburst\nvolcano'}

In [ ]:
text_sentence = "Unrest <mask> in the country ."

In [ ]:
get_all_predictions(text_sentence)

Unrest <mask> in the country .


{'bart': 'continues\nis\nhas\nand\nin',
 'bert': 'raged\nspread\nerupted\nreigned\nflared',
 'electra': 'continues\ncontinued\nremains\noccurred\nexists',
 'roberta': 's\ncontinues\nrising\npersists\ngrowing',
 'xlm': 'led\nling\ns\nlings\nle',
 'xlnet': 'in\nhere\nis\nIn\nthere'}

In [ ]:
text_sentence = "poison someone 's <mask> ."

In [ ]:
get_all_predictions(text_sentence)

poison someone 's <mask> .


{'bart': 'water\ndrink\nbrain\nmouth\nblood',
 'bert': 'liver\nblood\nwine\ndrink\nflesh',
 'electra': 'body\nmouth\nmind\nskin\nface',
 'roberta': 'life\nname\nhead\nblood\nsoul',
 'xlm': 'hair\nmouth\nblood\nlife\nhead',
 'xlnet': 's\nhouse\nplace\nhome\nbusiness'}

In [ ]:
text_sentence = "<mask> someone 's mind ."

In [ ]:
get_all_predictions(text_sentence)

<mask> someone 's mind .


{'bart': '3\n5\n4\nAn\nMake',
 'bert': 'probe\nread\nreading\ntouched\nin',
 'electra': 'in\non\nchange\nnot\nand',
 'roberta': 'in\nIn\non\nReading\nRead',
 'xlm': 'nut\nrepair\nJoin\nfulfill\nbomb',
 'xlnet': 'someone\ns\nin\nof\nto'}

In [ ]:
text_sentence = "<mask> were climbing after prices were lowered ."

In [ ]:
get_all_predictions(text_sentence)

<mask> were climbing after prices were lowered .


{'bart': 'C\nOil\nT\nPr\nU',
 'bert': 'prices\nsales\nrates\ncosts\nwages',
 'electra': 'prices\nrates\nthey\ntemperatures\nstocks',
 'roberta': 'Sales\nThey\nand\nstocks\nShares',
 'xlm': 'prices\nThey\ncosts\nWe\nParasites',
 'xlnet': 'they\nwho\nclimbing\nThey\nthat'}

In [ ]:
text_sentence = "Sales were <mask> after prices were lowered ."

In [ ]:
get_all_predictions(text_sentence)

Sales were <mask> after prices were lowered .


{'bart': 'also\ndown\nup\nslightly\nflat',
 'bert': 'booming\nincreased\nboost\nincreasing\nboosted',
 'electra': 'halted\nreduced\nclosed\nsuspended\nstopped',
 'roberta': 'steady\nup\nhigher\nflat\nstable',
 'xlm': 'increasing\nincreased\nhigh\nraised\ngrowing',
 'xlnet': 'until\nwere\nagain\ntoday\nwhere'}

In [ ]:
text_sentence = "This behavior will ruin your <mask> of winning the election ."

In [ ]:
get_all_predictions(text_sentence)

This behavior will ruin your <mask> of winning the election .


{'bart': 'chances\nchance\ncampaign\nparty\nhopes',
 'bert': 'chances\nchance\nprospects\nhopes\nhope',
 'electra': 'chances\nchance\nhopes\nhope\nrecord',
 'roberta': 'chances\nchance\nhopes\nprospects\nodds',
 'xlm': 'chances\nchance\nodds\nhope\npossibility',
 'xlnet': 'of\noff\nnot\nif\nfailing'}

In [ ]:
text_sentence = "This behavior will <mask> your chances of winning the election ."

In [ ]:
get_all_predictions(text_sentence)

This behavior will <mask> your chances of winning the election .


{'bart': 'not\nhelp\nincrease\ngreatly\nonly',
 'bert': 'boost\nincrease\nimprove\nruin\nhurt',
 'electra': 'increase\nimprove\nreduce\nboost\nenhance',
 'roberta': 'affect\ndecrease\nhurt\nreduce\nincrease',
 'xlm': 'increase\nimprove\nenhance\nlower\nboost',
 'xlnet': 'your\naffect\nmy\nbe\ntheir'}

In [ ]:
text_sentence = "The sales tax is <mask> into the state income tax ."

In [ ]:
get_all_predictions(text_sentence)

The sales tax is <mask> into the state income tax .


{'bart': 'not\nalso\nthen\ncollected\nincorporated',
 'bert': 'incorporated\nconverted\nintegrated\nmerged\ntransformed',
 'electra': 'converted\ndivided\nincorporated\nturned\ntranslated',
 'roberta': 'absorbed\nfolded\nrolled\nincorporated\nmerged',
 'xlm': 'added\nbuilt\nincluded\npaid\nallowed',
 'xlnet': 'into\nin\nout\nover\ngoing'}

In [ ]:
text_sentence = "The sales <mask> is absorbed into the state income tax ."

In [ ]:
get_all_predictions(text_sentence)

The sales <mask> is absorbed into the state income tax .


{'bart': 'tax\nand\ntaxes\nor\nof',
 'bert': 'tax\nrevenue\ntaxes\nincome\nsurplus',
 'electra': 'tax\nrevenue\nvalue\nrate\namount',
 'roberta': 'tax\nrevenue\ntaxes\nfee\nprice',
 'xlm': 'tax\nprice\nforce\ntaxes\nproduct',
 'xlnet': 'tax\nin\nprogram\nof\nis'}

In [ ]:
text_sentence = "The invaders spread their <mask> all over the country ."

In [ ]:
get_all_predictions(text_sentence)

The invaders spread their <mask> all over the country .


{'bart': 'evil\narmy\nwicked\ndeadly\nwar',
 'bert': 'influence\narms\nbombs\npower\nspread',
 'electra': 'weapons\nprey\nattacks\ntentacles\narms',
 'roberta': 'weapons\ndiseases\npropaganda\ntentacles\ndisease',
 'xlm': 'arms\npropaganda\nspread\nword\nway',
 'xlnet': 'all\nspread\nout\nway\nover'}

In [ ]:
text_sentence = "The invaders <mask> their language all over the country ."

In [ ]:
get_all_predictions(text_sentence)

The invaders <mask> their language all over the country .


{'bart': 'spread\nused\nleft\nwere\nhave',
 'bert': 'spread\nused\nspoke\nlost\nshouted',
 'electra': 'spread\nspeak\nspoken\nspoke\nread',
 'roberta': 'spread\nused\npracticed\nspoke\nintroduced',
 'xlm': 'spread\nextend\nspoke\nbroadcast\nsold',
 'xlnet': 'have\nput\nof\nare\ntook'}

In [ ]:
text_sentence = "Interns have to <mask> for a few months ."

In [ ]:
get_all_predictions(text_sentence)

Interns have to <mask> for a few months .


{'bart': 'stay\nbe\nlive\nwork\nattend',
 'bert': 'stay\nintern\nwait\nleave\nwork',
 'electra': 'stay\nwait\nleave\nwork\nsleep',
 'roberta': 'stay\nleave\nwait\nwork\nreturn',
 'xlm': 'wait\nwork\nsit\npay\nstay',
 'xlnet': 'go\nbe\nstay\nfor\nwait'}

In [ ]:
text_sentence = "They <mask> through the job candidates ."

In [ ]:
get_all_predictions(text_sentence)

They <mask> through the job candidates .


{'bart': 'are\nwill\nalso\nhave\nwere',
 'bert': 'pushed\npassed\nfiltered\nsorted\nran',
 'electra': 'go\nwent\nget\nrun\ngot',
 'roberta': 'went\ngo\nrun\nran\nsort',
 'xlm': 'run\ndug\nlooked\ncut\ntalk',
 'xlnet': 'went\nthrough\ngo\nway\nget'}

In [ ]:
text_sentence = "The dress <mask> her beautiful figure ."

In [ ]:
get_all_predictions(text_sentence)

The dress <mask> her beautiful figure .


{'bart': 'accent\nalso\nshowed\nperfectly\ncomple',
 'bert': 'hugged\nrevealed\nhighlighted\nshowcased\nframed',
 'electra': 'is\nwas\nfeatures\nshows\nfeatured',
 'roberta': 'hugged\nhighlighted\nemphasized\nhighlights\nreflected',
 'xlm': 'of\ned\nheld\nfed\nshows',
 'xlnet': 'her\nhis\nmy\nshe\nhe'}

In [ ]:
text_sentence = "<mask> the child with pride ."

In [ ]:
get_all_predictions(text_sentence)

<mask> the child with pride .


{'bart': '5\n4\n3\n10\n8',
 'bert': 'fill\nfills\nstroke\nfilled\nfilling',
 'electra': 'bless\ntreat\nlove\nserve\npride',
 'roberta': 'adding\nImagine\nSee\nWatch\nsaid',
 'xlm': 'treat\nraise\nfeed\nraised\nArm',
 'xlnet': 'the\nchild\nof\nlittle\na'}

In [ ]:
text_sentence = "The listeners <mask> when he discussed his strange ideas ."

In [ ]:
get_all_predictions(text_sentence)

The listeners <mask> when he discussed his strange ideas .


{'bart': 'were\nlaughed\nalso\nloved\nhad',
 'bert': 'laughed\ncringed\ngiggled\nlistened\nchuckled',
 'electra': 'laughed\nagreed\ngasped\ncheered\nsmiled',
 'roberta': 'laughed\ncheered\ngasped\nlistened\napplauded',
 'xlm': 'laugh\nsurprised\nexcited\npleased\nimpressed',
 'xlnet': 'miss\near\nlisteners\nthat\nears'}

In [ ]:
text_sentence = "The path <mask> all the way to the top of the hill ."

In [ ]:
get_all_predictions(text_sentence)

The path <mask> all the way to the top of the hill .


{'bart': 'is\ncontinues\nfollows\nruns\nleads',
 'bert': 'led\nclimbed\nwound\ncontinued\nstretched',
 'electra': 'goes\nruns\ncontinues\nleads\nis',
 'roberta': 'goes\ncontinues\nruns\nleads\nextends',
 'xlm': 'runs\nran\nrose\nis\nwent',
 'xlnet': 'all\nup\nback\ngoing\npath'}

In [ ]:
text_sentence = "<mask> the contents of a book into a summary ."

In [ ]:
get_all_predictions(text_sentence)

<mask> the contents of a book into a summary .


{'bart': 'Summary\nAn\nBook\nThis\nThe',
 'bert': 'transform\nturning\ntranslate\ncondensed\nturn',
 'electra': 'combine\ntranslate\nturn\ntranslates\nput',
 'roberta': 'Putting\nPut\nBreak\nBreaking\nTurn',
 'xlm': 'Turn\nturn\nPour\nRoll\nPull',
 'xlnet': 'the\nsubject\nsummary\nin\nparts'}

In [ ]:
text_sentence = "This poses an interesting <mask> ."
get_all_predictions(text_sentence)

This poses an interesting <mask> .


{'bart': 'question\nchallenge\nproblem\ndilemma\ncon',
 'bert': 'threat\nquestion\nproblem\nchallenge\nissue',
 'electra': 'challenge\nsituation\nissue\nmoment\npicture',
 'roberta': 'question\nproblem\nchallenge\nissue\ndilemma',
 'xlm': 'question\ndiscussion\nmessage\nchallenge\nsubject',
 'xlnet': 'a\nin\nfirst\nto\nor'}

##TroFi

In [ ]:
text_sentence = "Unfortunately , they wo n't work in the harbor tunnel where business travelers sometimes get <mask> on the way in from the airport . . . "

In [ ]:
get_all_predictions(text_sentence)

Unfortunately , they wo n't work in the harbor tunnel where business travelers sometimes get <mask> on the way in from the airport . . . 


{'bart': 'stuck\nlost\na\ninto\nhit',
 'bert': 'stuck\nlost\nstranded\nwrecked\nammed',
 'electra': 'off\nout\nlost\naway\nup',
 'roberta': 'stuck\nlost\nstranded\nhit\ncaught',
 'xlm': 'stuck\ncaught\nlost\nhurt\ntogether',
 'xlnet': 'lost\nget\nstuck\ndrowned\nout'}

In [ ]:
text_sentence = "It also vindicates Pratt 's efforts to <mask> problems in customer service that contributed to its loss of dominance in the commercial jet engine field to General Electric Co"

In [ ]:
get_all_predictions(text_sentence)

It also vindicates Pratt 's efforts to <mask> problems in customer service that contributed to its loss of dominance in the commercial jet engine field to General Electric Co


{'bart': 'repair\nrebuild\nimprove\nfix\novercome',
 'bert': 'solve\nfix\nresolve\naddress\novercome',
 'electra': 'solve\naddress\novercome\nfix\nresolve',
 'roberta': 'fix\naddress\ncorrect\nresolve\nremedy',
 'xlm': 'address\nrepair\ncorrect\nsolve\nfix',
 'xlnet': 'solve\nto\naddress\nfix\nthe'}

In [ ]:
text_sentence = "Weeks later , '' Penthouse finally realized that Dominion was intentionally <mask> the deal , '' sought other financing , failed , and had to abandon the project , the judge said "

In [ ]:
get_all_predictions(text_sentence)

Weeks later , '' Penthouse finally realized that Dominion was intentionally <mask> the deal , '' sought other financing , failed , and had to abandon the project , the judge said 


{'bart': 'trying\nmisleading\nlying\nwithholding\nhiding',
 'bert': 'ruining\nblocking\npushing\ndamaging\ndestroying',
 'electra': 'blocking\nbreaking\nabandoning\nruining\nviolating',
 'roberta': 'rigging\nbreaking\nkilling\nmanipulating\ncutting',
 'xlm': 'funding\nending\nbuying\nmaking\nbuilding',
 'xlnet': 'in\nbehind\ninto\non\nfinancing'}

In [ ]:
text_sentence = "After years of being out of work and on welfare , many of the unemployed now lack the ability or initiative to <mask> openings "

In [ ]:
get_all_predictions(text_sentence)

After years of being out of work and on welfare , many of the unemployed now lack the ability or initiative to <mask> openings 


{'bart': 'find\nlook\ntake\nkeep\nget',
 'bert': 'seek\nmake\npursue\nfight\nfill',
 'electra': 'find\nmake\nget\ncreate\nreach',
 'roberta': 'fill\nfind\nseek\ncreate\nmake',
 'xlm': 'open\nsecure\nfill\nmake\naccept',
 'xlnet': 'to\njob\nmake\nopen\naccess'}

In [ ]:
text_sentence = "She 's been able to <mask> to this policy , largely because of the nature of her music "

In [ ]:
get_all_predictions(text_sentence)

She 's been able to <mask> to this policy , largely because of the nature of her music 


{'bart': 'maintain\nremain\navoid\nhold\ncontinue',
 'bert': 'adhere\nstick\nconform\nadapt\ncater',
 'electra': 'adhere\nstick\nadapt\nconform\nagree',
 'roberta': 'adhere\nstick\nadapt\nconform\nappeal',
 'xlm': 'conform\nagree\nrespond\nadjust\nadapt',
 'xlnet': 'come\nget\nto\ngo\naccess'}

In [ ]:
text_sentence = "The Soviet Union is unwilling and unlikely to <mask> the deadline for withdrawing all its troops from Afghanistan , despite its latest blustering and military maneuvering there , Western and Soviet officials indicated "

In [ ]:
get_all_predictions(text_sentence)

The Soviet Union is unwilling and unlikely to <mask> the deadline for withdrawing all its troops from Afghanistan , despite its latest blustering and military maneuvering there , Western and Soviet officials indicated 


{'bart': 'extend\nmeet\nchange\npostpone\ndelay',
 'bert': 'meet\nmiss\naccept\nface\nextend',
 'electra': 'meet\nreach\nset\nmiss\npass',
 'roberta': 'meet\nmiss\nextend\ndelay\naccept',
 'xlm': 'meet\nreach\npass\nfulfill\nhit',
 'xlnet': 'meet\nignore\naccept\nto\navoid'}

In [ ]:
text_sentence = "'' I found myself <mask> out on the steps in boot camp , '' he recalls "

In [ ]:
get_all_predictions(text_sentence)

'' I found myself <mask> out on the steps in boot camp , '' he recalls 


{'bart': 'in\ncrying\ngetting\nwith\non',
 'bert': 'passed\nsprawled\nhanging\ncrying\nstretched',
 'electra': 'hanging\ngoing\nwalking\ncamping\nlaying',
 'roberta': 'passed\nhanging\ncrying\nsitting\ncamping',
 'xlm': 'sitting\nrunning\ncoming\nright\ndug',
 'xlnet': 'yourself\nway\nout\nback\nmyself'}

In [ ]:
text_sentence = "Last month , Thiokol said it settled the last suit with the estates of the seven crew members <mask> in the Challenger "

In [ ]:
get_all_predictions(text_sentence)

Last month , Thiokol said it settled the last suit with the estates of the seven crew members <mask> in the Challenger 


{'bart': 'who\nkilled\nof\nand\ninvolved',
 'bert': 'involved\nlost\nkilled\naboard\nremaining',
 'electra': 'involved\nworking\nliving\nremaining\noperating',
 'roberta': 'killed\ninvolved\ninjured\nlost\nmissing',
 'xlm': 'involved\nsitting\nworking\nliving\noperating',
 'xlnet': 'involved\nin\nkilled\nwho\ninjured'}

In [ ]:
text_sentence = "Incumbent Spyros Kyprianou was <mask> out in voting last week"

In [ ]:
get_all_predictions(text_sentence)

Incumbent Spyros Kyprianou was <mask> out in voting last week


{'bart': 'the\nnarrowly\nalso\ndeclared\nforced',
 'bert': 'voted\nstill\nknocked\nleft\nwashed',
 'electra': 'knocked\nruled\nwiped\nshut\nnot',
 'roberta': 'shut\nedged\nknocked\nwiped\nvoted',
 'xlm': 'carried\nleft\nhelped\nnot\nright',
 'xlnet': 'was\nout\nalso\nnot\nhimself'}

In [ ]:
text_sentence = "Those progressive cities now building or planning transit systems for the next century are not viewing the scene from an ivory tower but realize , in a practical sense , that better transit is vital not only for the millions of Americans who currently <mask> onto a transit bus or train over 8.3 billion times each year , but also to their future citizens as well "
get_all_predictions(text_sentence)

Those progressive cities now building or planning transit systems for the next century are not viewing the scene from an ivory tower but realize , in a practical sense , that better transit is vital not only for the millions of Americans who currently <mask> onto a transit bus or train over 8.3 billion times each year , but also to their future citizens as well 


{'bart': 'use\nrely\nlive\ncommute\nride',
 'bert': 'step\nhop\nlog\nload\nflock',
 'electra': 'get\nride\nlog\ngo\njump',
 'roberta': 'step\nhop\nget\njump\npile',
 'xlm': 'turn\nhop\ntravel\nhead\ngo',
 'xlnet': 'hop\nonto\nhold\nride\ngrab'}

In [ ]:
text_sentence = "Seagram must decide by Feb. 9 whether to come up with a higher offer that would <mask> Grand Met out of the bidding war and frustrate its expansion strategy "
get_all_predictions(text_sentence)

Seagram must decide by Feb. 9 whether to come up with a higher offer that would <mask> Grand Met out of the bidding war and frustrate its expansion strategy 


{'bart': 'knock\nforce\nkeep\ntake\nleave',
 'bert': 'make\ncut\nhelp\nget\nbe',
 'electra': 'be\nget\nmake\nkeep\nhave',
 'roberta': 'lock\nknock\ntake\nkeep\nshut',
 'xlm': 'take\npush\ndrive\nbring\npull',
 'xlnet': 'pull\nkeep\ndraw\nwithdraw\nthrow'}

In [ ]:
text_sentence = "Charity officials say they need trustees who are willing to <mask> up their sleeves and work for the organization and who will back up their commitment with donations "
get_all_predictions(text_sentence)

Charity officials say they need trustees who are willing to <mask> up their sleeves and work for the organization and who will back up their commitment with donations 


{'bart': 'roll\nwork\nput\nboth\nsit',
 'bert': 'roll\nclimb\npuff\nlift\nride',
 'electra': 'roll\ngive\ntie\ncut\nback',
 'roberta': 'roll\nopen\nrolled\nzip\npush',
 'xlm': 'take\ngive\nroll\nsit\npull',
 'xlnet': 'go\nput\ntake\ncome\nup'}

In [ ]:
text_sentence = "This provides greatly enhanced trouble - shooting capabilities when something does go wrong , and can even be used to automatically switch in a good spare card if something does <mask> "
get_all_predictions(text_sentence)

This provides greatly enhanced trouble - shooting capabilities when something does go wrong , and can even be used to automatically switch in a good spare card if something does <mask> 


{'bart': 'go\nhappen\nnot\nbreak\nindeed',
 'bert': 'fail\nhappen\nwrong\nmiss\nbreak',
 'electra': 'happen\ngo\nwork\nwrong\nnot',
 'roberta': 'go\nhappen\nget\noccur\nbreak',
 'xlm': 'wrong\ngo\noccur\nhappen\nslip',
 'xlnet': 'go\nhappen\nnot\ndo\nsomething'}

In [ ]:
text_sentence = "The Japanese carrier was shaken in 1985 by a crash of a JAL Boeing 747 that <mask> 520 people , the worst single - aircraft accident in aviation history , and safety is a central issue in JAL \'s current negotiations with several unions"
get_all_predictions(text_sentence)

The Japanese carrier was shaken in 1985 by a crash of a JAL Boeing 747 that <mask> 520 people , the worst single - aircraft accident in aviation history , and safety is a central issue in JAL 's current negotiations with several unions


{'bart': 'killed\nclaimed\ndestroyed\ntook\nleft',
 'bert': 'killed\ninjured\nlost\nmurdered\ncrushed',
 'electra': 'killed\ninjured\nwounded\nleft\naffected',
 'roberta': 'killed\ninjured\nclaimed\nhurt\nkilling',
 'xlm': 'killed\ninvolved\nhurt\nlost\ntook',
 'xlnet': 'killed\ninjured\nclaimed\nkilling\ndied'}

In [ ]:
text_sentence = "It is \'\' Hotlanta \'\' : capital city of the South at a time when the South has risen ; city of myriad services at a time when the service economy rules ; city of the 1980s , more suburban than downtown , more footloose than <mask>"
get_all_predictions(text_sentence)

It is '' Hotlanta '' : capital city of the South at a time when the South has risen ; city of myriad services at a time when the service economy rules ; city of the 1980s , more suburban than downtown , more footloose than <mask>


{'bart': 'the\ndowntown\nupt\ncity\nhip',
 'bert': 'ever\ndowntown\nbust\nanything\nshri',
 'electra': 'downtown\nsuburban\nuptown\noutside\ntoday',
 'roberta': 'downtown\nurban\nrural\ncity\nDowntown',
 'xlm': 'capital\nwater\nsouth\npoverty\ncountry',
 'xlnet': 'downtown\nsuburban\never\ncity\nDowntown'}

In [ ]:
text_sentence = "In Louisiana , CIT was <mask> with repossessed ships that had supplied drilling platforms in the Gulf of Mexico "
get_all_predictions(text_sentence)

In Louisiana , CIT was <mask> with repossessed ships that had supplied drilling platforms in the Gulf of Mexico 


{'bart': 'the\nresponsible\ncharged\ninvolved\nalso',
 'bert': 'dealing\nworking\ncrawling\nfighting\nfilled',
 'electra': 'equipped\narmed\nassociated\nstocked\ncombined',
 'roberta': 'working\ndealing\noperating\ncompeting\nflooded',
 'xlm': 'associated\ninvolved\nregistered\ndealing\nalong',
 'xlnet': 'involved\ndealing\nin\nnot\nwas'}

In [ ]:
text_sentence = "Orion , a contract carrier which formerly <mask> for United Parcel Service , had agreed to replace Eastern pilots if they followed the machinists on a walkout "
get_all_predictions(text_sentence)

Orion , a contract carrier which formerly <mask> for United Parcel Service , had agreed to replace Eastern pilots if they followed the machinists on a walkout 


{'bart': 'flew\noperated\nworked\nhandled\nserved',
 'bert': 'worked\ncompeted\noperated\nsailed\nflew',
 'electra': 'worked\noperated\nran\nwas\nserved',
 'roberta': 'worked\nflew\noperated\nsubstituted\ncompeted',
 'xlm': 'worked\nran\nworks\npaid\nregistered',
 'xlnet': 'worked\noperated\nworks\narranged\ncontracted'}

In [ ]:
text_sentence = "But if all sides discover self - interest in moving toward effective control of the toxics problem , instead of <mask> grimly to their familiar trenches , the results will be as remarkable as the law itself "
get_all_predictions(text_sentence)

But if all sides discover self - interest in moving toward effective control of the toxics problem , instead of <mask> grimly to their familiar trenches , the results will be as remarkable as the law itself 


{'bart': 'returning\nretreating\njust\nsimply\ncontinuing',
 'bert': 'clinging\nsticking\nholding\nmarching\nclutching',
 'electra': 'returning\nmoving\nclinging\nretreating\ngoing',
 'roberta': 'retreating\nreturning\nretiring\nsinking\nclinging',
 'xlm': 'adding\ngiving\ngoing\ndriving\ncoming',
 'xlnet': 'clinging\nreturning\nstuck\nsticking\nattending'}

In [ ]:
text_sentence = "Real power still <mask> with Pretoria just as it does with Jerusalem"
get_all_predictions(text_sentence)

Real power still <mask> with Pretoria just as it does with Jerusalem


{'bart': 'resides\nrests\nlies\nin\nlives',
 'bert': 'rests\nsits\nstruggles\nresides\nlies',
 'electra': 'works\nexists\nplays\nis\nstays',
 'roberta': 'rests\nlies\nresides\nexists\nsits',
 'xlm': 's\nhas\nruns\nalong\ned',
 'xlnet': 'still\nis\nremains\ntogether\nalive'}

In [ ]:
text_sentence = "Every time it <mask> , the family placed cups and bowls all over the house to catch the leaks"
get_all_predictions(text_sentence)

Every time it <mask> , the family placed cups and bowls all over the house to catch the leaks


{'bart': 'r\nrains\nwas\nleaked\nstarted',
 'bert': 'rained\nhappened\nstarted\noccurred\nbegan',
 'electra': 'happened\nleaked\ndid\nrained\nworked',
 'roberta': 'leaked\nrains\nhappened\nbroke\nleaks',
 'xlm': 'happened\nopened\nwas\nstopped\nstarted',
 'xlnet': 'was\nit\nis\nIt\nthey'}

In [ ]:
text_sentence = "And Mr. McGegan , who does n\'t mince words when he dislikes something or someone -LRB- he called the countertenor in his recent Long Beach production \'\' awful \'\' -RRB- , said he was <mask> out by Mr. Davies \'s strangely articulated yet strikingly clear conducting of Beethoven \'s \'\' Eroica \'\' Symphony"
get_all_predictions(text_sentence)

And Mr. McGegan , who does n't mince words when he dislikes something or someone -LRB- he called the countertenor in his recent Long Beach production '' awful '' -RRB- , said he was <mask> out by Mr. Davies 's strangely articulated yet strikingly clear conducting of Beethoven 's '' Eroica '' Symphony


{'bart': 'impressed\nnot\nstruck\npleased\nparticularly',
 'bert': 'grossed\nfreaked\ndrowned\nput\ntuned',
 'electra': 'freaked\ndrowned\nsingled\nstruck\nstressed',
 'roberta': 'put\nstressed\nturned\ncaught\nworn',
 'xlm': 'let\nhelped\nstood\nleft\ncarried',
 'xlnet': 'put\nleft\nstruck\nturned\nthrown'}

In [ ]:
text_sentence = "London share prices finished a bumpy session little changed Monday , after the market <mask> on news of an unexpected rise in U.K. interest rates"
get_all_predictions(text_sentence)

London share prices finished a bumpy session little changed Monday , after the market <mask> on news of an unexpected rise in U.K. interest rates


{'bart': 'had\nopened\ntook\nfell\nrallied',
 'bert': 'exploded\nseized\nburst\nchoked\nturned',
 'electra': 'fell\nclosed\nrose\nrallied\nplunged',
 'roberta': 'rallied\nplunged\njumped\nfell\nslumped',
 'xlm': 'turned\nreported\ntook\nwent\nwas',
 'xlnet': 'market\nclosed\nput\nwas\nwent'}

In [ ]:
text_sentence = "The virus <mask> Argonne National Laboratory outside Chicago starting at 11:54 p.m. EST Wednesday and throughout the night"
get_all_predictions(text_sentence)

The virus <mask> Argonne National Laboratory outside Chicago starting at 11:54 p.m. EST Wednesday and throughout the night


{'bart': 'will\nwas\nis\ncould\ntesting',
 'bert': 'infected\nattacked\nhit\nstruck\nentered',
 'electra': 'reached\nleft\nvisited\nentered\nhit',
 'roberta': 'struck\nhit\nreached\nentered\nattacked',
 'xlm': 'of\nreached\nhas\nsank\nwill',
 'xlnet': 'at\nAt\nof\nin\non'}

##TroFi-X

In [ ]:
text_sentence = 'When she hears that prisoners sometimes carve their names into logs that float downriver , she grabs her <mask> and runs with her to the river . '
get_all_predictions(text_sentence)

When she hears that prisoners sometimes carve their names into logs that float downriver , she grabs her <mask> and runs with her to the river . 


{'bart': 'friend\nsister\nmother\ndaughter\nlittle',
 'bert': 'purse\ndaughter\nsister\nbackpack\nhand',
 'electra': 'hand\narm\nbag\nhead\ngun',
 'roberta': 'daughter\nson\ndog\nsister\nfriend',
 'xlm': 'cock\nhand\narms\nsuit\nclothes',
 'xlnet': 'head\nup\ncoat\ngun\nhand'}

In [ ]:
text_sentence = "The latest truce in the week-old battle , which has killed <mask> and injured 534 , was arranged by Syrian President Assad and Iranian President Khamenei ."
get_all_predictions(text_sentence)

The latest truce in the week-old battle , which has killed <mask> and injured 534 , was arranged by Syrian President Assad and Iranian President Khamenei .


{'bart': 'at\nmore\nover\nnearly\nsome',
 'bert': '235\n300\n600\n260\n261',
 'electra': 'thousands\nhundreds\n50\n40\n60',
 'roberta': 'hundreds\ndozens\nthousands\n130\n235',
 'xlm': 'four\ntwo\nthree\nmany\n100',
 'xlnet': 'and\nkilled\n43\n5\ndied'}

In [ ]:
text_sentence = "All agree that the state must diversify its industrial <mask> to escape the recently detrimental reliance on oil , said Ron Jones , chairman of the New Orleans Business Council and president of Louisiana Coca Cola Bottling Ltd . "
get_all_predictions(text_sentence)

All agree that the state must diversify its industrial <mask> to escape the recently detrimental reliance on oil , said Ron Jones , chairman of the New Orleans Business Council and president of Louisiana Coca Cola Bottling Ltd . 


{'bart': 'base\nand\nsector\neconomy\nproduction',
 'bert': 'resources\nsector\nproduction\nbase\noutput',
 'electra': 'operations\nization\nsector\ninfrastructure\noutput',
 'roberta': 'base\nsector\nsectors\nmix\neconomy',
 'xlm': 'capital\ncapacity\nresources\nlines\nsector',
 'xlnet': 'industry\nproduction\nsector\nindustries\nbusiness'}

In [ ]:
text_sentence = "A surprisingly strong October employment report , implying that the economy is n\'t <mask> off as fast as many investors had hoped , could force a sharp retreat in stock prices today , he said "
get_all_predictions(text_sentence)

A surprisingly strong October employment report , implying that the economy is n't <mask> off as fast as many investors had hoped , could force a sharp retreat in stock prices today , he said 


{'bart': 'cooling\nslowing\nreally\ntaking\nbeing',
 'bert': 'taking\ncooling\ngoing\nstarting\nspinning',
 'electra': 'paying\ncoming\ngoing\nbuying\nslowing',
 'roberta': 'cooling\ntaking\neasing\ncoming\nfalling',
 'xlm': 'paying\ncoming\ngoing\nturning\ntaking',
 'xlnet': 'quite\nreally\ngetting\ncoming\npulling'}

In [ ]:
text_sentence = "`` It floods the <mask> instead of having hot pinpoints of glaring light . \'\' "
get_all_predictions(text_sentence)

`` It floods the <mask> instead of having hot pinpoints of glaring light . '' 


{'bart': 'room\nwhole\nentire\narea\nscreen',
 'bert': 'room\nair\nsky\ntheater\nrooms',
 'electra': 'world\ncity\nsky\nwater\nair',
 'roberta': 'room\nsky\nscene\nscreen\nworld',
 'xlm': 'water\nriver\nair\nocean\nroom',
 'xlnet': 'the\ninstead\nin\nother\nrather'}

In [ ]:
text_sentence = "Next month , A & P will roll out these <mask> and jellies , now sold only in New York , to its 986 stores in the U.S . "
get_all_predictions(text_sentence)

Next month , A & P will roll out these <mask> and jellies , now sold only in New York , to its 986 stores in the U.S . 


{'bart': 'new\nand\nproducts\nflavors\nfresh',
 'bert': 'fruits\nbooks\nwines\nproducts\ncomics',
 'electra': 's\napples\njeans\nrolls\nnuts',
 'roberta': 'jams\ncookies\ncakes\njuices\nshakes',
 'xlm': 'butter\nflowers\ncookies\npies\nchips',
 'xlnet': 'cookies\nbars\nthese\nfruits\nhearts'}

In [ ]:
text_sentence = "-- The justices refused to hear an antitrust appeal by actor Paul Newman and director George Roy Hill in a suit charging that MCA Inc. and its Universal City Studios Inc. unit conspired to fix the <mask> of film revenue paid to artists ."
get_all_predictions(text_sentence)

-- The justices refused to hear an antitrust appeal by actor Paul Newman and director George Roy Hill in a suit charging that MCA Inc. and its Universal City Studios Inc. unit conspired to fix the <mask> of film revenue paid to artists .


{'bart': 'rates\nprice\nrate\nprices\nmarket',
 'bert': 'amount\npercentage\nlevel\nstream\ncut',
 'electra': 'amount\nbulk\nlevel\nbalance\nprice',
 'roberta': 'distribution\namount\nrate\npercentage\nbalance',
 'xlm': 'amount\ncost\nbalance\nflow\nprice',
 'xlnet': 'source\namount\ndistribution\nflow\nshare'}

In [ ]:
text_sentence = "Indiana Rep. Andrew Jacobs , a Democrat who \'s neutral in the presidential-nomination contest , says Mr. Gephardt reminds him of a time in his life when he had to eat a <mask> of dinners at a drive-in restaurant . "
get_all_predictions(text_sentence)

Indiana Rep. Andrew Jacobs , a Democrat who 's neutral in the presidential-nomination contest , says Mr. Gephardt reminds him of a time in his life when he had to eat a <mask> of dinners at a drive-in restaurant . 


{'bart': 'lot\ncouple\nmeal\nwhole\nvariety',
 'bert': 'couple\nbunch\nlot\nvariety\nton',
 'electra': 'couple\nlot\nbunch\nhandful\nvariety',
 'roberta': 'lot\ncouple\nseries\nvariety\nnumber',
 'xlm': 'lot\nvariety\ncouple\nnumber\ngroup',
 'xlnet': 'lot\nseries\ncouple\nbunch\ngroup'}

In [ ]:
text_sentence = "The result: Some car makers are cooling their hot <mask> ."
get_all_predictions(text_sentence)

The result: Some car makers are cooling their hot <mask> .


{'bart': 'engines\nwheels\nair\ncars\nengine',
 'bert': 'cars\nengines\nwheels\noil\nbodies',
 'electra': 'air\nsprings\nwater\ncars\ndogs',
 'roberta': 'spots\nrods\nheels\nwheels\nmarkets',
 'xlm': 'water\nsauce\nrod\nair\noil',
 'xlnet': 'hot\ncold\nheat\ncooling\ncool'}

In [ ]:
text_sentence = "While Mr. Gorbachev preached togetherness yesterday , Romania was kicking Hungary \'s <mask> out of the country . "
get_all_predictions(text_sentence)

While Mr. Gorbachev preached togetherness yesterday , Romania was kicking Hungary 's <mask> out of the country . 


{'bart': 'ambassador\nmost\nforeign\nprime\nformer',
 'bert': 'government\ncommunists\ntroops\npeople\narmy',
 'electra': 'head\nway\nback\npeople\nass',
 'roberta': 'ambassador\nCommunists\ndiplomats\npeople\nleaders',
 'xlm': 'arms\ngovernment\npeople\ncapital\nway',
 'xlnet': 's\nUnion\nway\n’\nown'}

In [ ]:
text_sentence = "Mr. Martineau also says the tractors reflect another Russian approach to manufacturing: `` You could melt down one <mask> and make three American tractors with the amount of metal the Russians have put in it , \'\' he says . "
get_all_predictions(text_sentence)

Mr. Martineau also says the tractors reflect another Russian approach to manufacturing: `` You could melt down one <mask> and make three American tractors with the amount of metal the Russians have put in it , '' he says . 


{'bart': 'of\nAmerican\nRussian\npiece\ntractor',
 'bert': 'tractor\nengine\nmachine\nplant\ntree',
 'electra': 'car\nmachine\ntruck\ntractor\none',
 'roberta': 'tractor\ncar\nplant\nfactory\nthing',
 'xlm': 'Russian\nAmerican\ngun\nGerman\ntruck',
 'xlnet': 'tractor\nengine\ntruck\nfactory\nmachine'}

In [ ]:
text_sentence = "But there were stumbling <mask> from the start . "
get_all_predictions(text_sentence)

But there were stumbling <mask> from the start . 


{'bart': 'blocks\nsteps\nstones\nblock\nmoments',
 'bert': 'blocks\nsteps\nerrors\nstones\ns',
 'electra': 's\nblocks\nproblems\naround\npatterns',
 'roberta': 'blocks\npoints\nstones\nbumps\nsteps',
 'xlm': 's\nquestions\nthrough\nalong\nproblems',
 'xlnet': 'from\nabout\nright\ndown\naround'}

In [ ]:
text_sentence = "`` A strong opening in platinum coupled with a weak dollar just grabbed <mask> by the horns and took it on up , \'\' he said . "
get_all_predictions(text_sentence)

`` A strong opening in platinum coupled with a weak dollar just grabbed <mask> by the horns and took it on up , '' he said . 


{'bart': 'the\nthis\nit\na\nthat',
 'bert': 'platinum\nit\ngold\nmoney\nbusiness',
 'electra': 'me\nyou\nit\nus\nhim',
 'roberta': 'gold\nit\nplatinum\nsilver\nmetal',
 'xlm': 'it\nhim\nthem\nher\nme',
 'xlnet': 'grabbed\nby\ngrab\nit\nup'}

In [ ]:
text_sentence = "`` I \'m quite sure there was someone within the IBM structure that could have just reached in and fixed <mask> . \'\' "
get_all_predictions(text_sentence)

`` I 'm quite sure there was someone within the IBM structure that could have just reached in and fixed <mask> . '' 


{'bart': 'it\nthe\nthis\nthat\nwhat',
 'bert': 'it\nproblems\neverything\nthings\nbugs',
 'electra': 'it\nthat\nthem\nme\neverything',
 'roberta': 'it\nthat\nthis\neverything\nthings',
 'xlm': 'it\nher\nup\nthis\nthem',
 'xlnet': 'the\na\nthis\nthat\nan'}

In [ ]:
text_sentence = "Nevertheless , they continue to pump <mask> into the market . "
get_all_predictions(text_sentence)

Nevertheless , they continue to pump <mask> into the market . 


{'bart': 'billions\nmore\na\nmoney\nthe',
 'bert': 'money\ncash\nproducts\ndrugs\nstock',
 'electra': 'money\nback\ncash\nheavily\nwater',
 'roberta': 'money\ncash\nliquidity\nproducts\nfunds',
 'xlm': 'water\noil\npeople\nit\nthem',
 'xlnet': 'further\nback\nup\nin\ninto'}

In [ ]:
text_sentence = "During the past 18 months , the Houston-based unit of Texas Air Corp. absorbed several <mask> , in the process inheriting a hodge-podge of different aircraft . "
get_all_predictions(text_sentence)

During the past 18 months , the Houston-based unit of Texas Air Corp. absorbed several <mask> , in the process inheriting a hodge-podge of different aircraft . 


{'bart': 'of\nsmaller\nother\nsmall\nairlines',
 'bert': 'aircraft\nsquadrons\ncompanies\nunits\npilots',
 'electra': 'aircraft\nplanes\njets\ncompanies\nships',
 'roberta': 'airlines\nplanes\nsubsidiaries\naircraft\ncarriers',
 'xlm': 'parasites\nproducts\nwater\nresources\nfish',
 'xlnet': 'persons\nmany\nhundred\npeople\nmore'}

In [ ]:
text_sentence = "`` I do n\'t think anyone liked the Nissan campaign , but they went out of their way not to knock <mask> , \'\' says Eli Bloom , the New York GM dealer who sits on the selection committee . -LRB- Mr . "
get_all_predictions(text_sentence)

`` I do n't think anyone liked the Nissan campaign , but they went out of their way not to knock <mask> , '' says Eli Bloom , the New York GM dealer who sits on the selection committee . -LRB- Mr . 


{'bart': 'the\nGM\non\nus\nout',
 'bert': 'around\nout\nyou\nhard\nit',
 'electra': 'it\nout\ndown\nthem\nback',
 'roberta': 'it\nus\nthem\nanybody\nme',
 'xlm': 'them\nit\nhim\ndown\nyou',
 'xlnet': 'it\nthem\nus\nanybody\nme'}

In [ ]:
text_sentence = "`` That remark -LRB- of the official -RRB- definitely cooled the <mask> of the bulls , \'\' said Paul Clohesy , chief dealer at Australia and New Zealand Banking Group Ltd. in New York . "
get_all_predictions(text_sentence)

`` That remark -LRB- of the official -RRB- definitely cooled the <mask> of the bulls , '' said Paul Clohesy , chief dealer at Australia and New Zealand Banking Group Ltd. in New York . 


{'bart': 'fire\nenthusiasm\nmarket\nmood\nappetite',
 'bert': 'hearts\nenthusiasm\nmood\nspirits\nblood',
 'electra': 'spirits\nminds\nblood\nbutt\nmood',
 'roberta': 'appetite\nenthusiasm\ntemper\nmood\npulse',
 'xlm': 'weight\nwater\nenergy\nprice\nflavor',
 'xlnet': 'temper\nhead\ntail\nspirits\nfeet'}

In [ ]:
text_sentence = "The subject is Oedipus \'s death , and perhaps because death is the one thing that still fills <mask> with awe , the excellence of this scene transmits a shiver of the tragedy \'s original force . "
get_all_predictions(text_sentence)

The subject is Oedipus 's death , and perhaps because death is the one thing that still fills <mask> with awe , the excellence of this scene transmits a shiver of the tragedy 's original force . 


{'bart': 'the\nus\na\nShakespeare\nthis',
 'bert': 'me\nus\nyou\npeople\none',
 'electra': 'me\nus\nyou\nit\none',
 'roberta': 'us\nme\nhim\none\nyou',
 'xlm': 'us\npeople\nhim\nme\nyou',
 'xlnet': 'us\nme\none\nyou\nhumanity'}

In [ ]:
text_sentence = "The issue is particularly hot in Florida , where a coalition of civic leaders , civil libertarians and Hispanic groups went to federal court recently in an unsuccessful attempt to knock the proposed <mask> off the ballot . "
get_all_predictions(text_sentence)

The issue is particularly hot in Florida , where a coalition of civic leaders , civil libertarians and Hispanic groups went to federal court recently in an unsuccessful attempt to knock the proposed <mask> off the ballot . 


{'bart': 'constitutional\nban\namendment\nlaw\nlanguage',
 'bert': 'bill\nlegislation\namendment\ncandidate\nconstitution',
 'electra': 'bill\nconstitution\nissue\nproposal\nlegislation',
 'roberta': 'measure\namendment\nreferendum\ntax\nmeasures',
 'xlm': 'bill\ntax\nreform\nreferendum\nlaw',
 'xlnet': 'Florida\nlaw\ninitiative\nreferendum\nstate'}

In [ ]:
text_sentence = "Three former scholarship winners now fly border <mask> for the U.S. Customs Service . "
get_all_predictions(text_sentence)

Three former scholarship winners now fly border <mask> for the U.S. Customs Service . 


{'bart': 'patrol\npatrols\ncontrol\nsecurity\nguards',
 'bert': 'checks\npatrols\ncrossings\nmissions\npatrol',
 'electra': 'flights\nlines\nsigns\nroutes\nvisas',
 'roberta': 'patrol\nsecurity\npatrols\nchecks\nguards',
 'xlm': 's\ned\nline\nareas\nlines',
 'xlnet': 'for\nto\nhere\nin\nwith'}

In [ ]:
text_sentence = "There was market speculation yesterday that senior officials of Olympia & York may have met secretly with Michael D. Dingman , chairman and chief executive officer of Henley , in an effort to smooth over a <mask> between the two one-time suitors for Santa Fe , a Chicago-based railroad and energy concern . "
get_all_predictions(text_sentence)

There was market speculation yesterday that senior officials of Olympia & York may have met secretly with Michael D. Dingman , chairman and chief executive officer of Henley , in an effort to smooth over a <mask> between the two one-time suitors for Santa Fe , a Chicago-based railroad and energy concern . 


{'bart': 'potential\ndispute\npossible\nrift\nrecent',
 'bert': 'conversation\nrift\ndispute\nrivalry\nconflict',
 'electra': 'dispute\nrelationship\nrift\ngap\ndeal',
 'roberta': 'dispute\ndisagreement\nfeud\nspat\nrift',
 'xlm': 'dispute\ntrade\nconflict\ndebate\nrelationship',
 'xlnet': 'dispute\nrow\nconflict\ndisagreement\ndifferences'}

In [ ]:
text_sentence = "Karen Stern has played the <mask> surrounded by ice sculptures at a restaurant and as part of the display for a three-foot-high cake at a pastry competition  "
get_all_predictions(text_sentence)

Karen Stern has played the <mask> surrounded by ice sculptures at a restaurant and as part of the display for a three-foot-high cake at a pastry competition  


{'bart': 'role\nviolin\npart\npiano\nice',
 'bert': 'role\nviolin\npart\npiano\nfiddle',
 'electra': 'role\nstage\ngame\npiano\npart',
 'roberta': 'violin\npiano\nguitar\npart\nrole',
 'xlm': 'cake\nhorse\ncourt\nball\nevent',
 'xlnet': 'game\nstage\nfield\ntable\nturn'}

In [ ]:
text_sentence = "The Pac Man defense is named after the video game in which characters try to eat other characters before they get eaten <mask> . "
get_all_predictions(text_sentence)

The Pac Man defense is named after the video game in which characters try to eat other characters before they get eaten <mask> . 


{'bart': 'by\nthemselves\nand\ntoo\nor',
 'bert': 'alive\nfirst\nwhole\nquickly\ncompletely',
 'electra': 'up\nagain\nout\nanyway\noff',
 'roberta': 'by\ntoo\nagain\nthemselves\n...',
 'xlm': 'away\nup\nout\noff\ntogether',
 'xlnet': 'by\nalive\nand\nin\nback'}

In [ ]:
text_sentence = "Unself-consciously , the littlest cast member with the big voice steps into the audience in one number to open her wide cat-eyes and throat to melt the <mask> of one lucky patron each night . "
get_all_predictions(text_sentence)

Unself-consciously , the littlest cast member with the big voice steps into the audience in one number to open her wide cat-eyes and throat to melt the <mask> of one lucky patron each night . 


{'bart': 'hearts\nheart\nfrozen\nice\neyes',
 'bert': 'heart\nhearts\nbones\nskin\ninsides',
 'electra': 'heart\neyes\nmind\nblood\nbody',
 'roberta': 'heart\nhearts\nsmile\nsoul\nice',
 'xlm': 'blood\ncock\nheart\nmouth\nmilk',
 'xlnet': 'heart\nhearts\neyes\nthroat\nears'}

In [ ]:
text_sentence = "They have had to learn through the press about the sweet terms he had struck <mask> himself and his small management group "
get_all_predictions(text_sentence)

They have had to learn through the press about the sweet terms he had struck <mask> himself and his small management group 


{'bart': 'with\nfor\nin\nto\non',
 'bert': 'between\nwith\nfor\nagainst\nto',
 'electra': 'for\non\nbetween\nwith\nupon',
 'roberta': 'between\nwith\nfor\non\namong',
 'xlm': 'by\nupon\nwith\nbetween\nfor',
 'xlnet': 'himself\nupon\nstruck\nin\nbetween'}

In [ ]:
text_sentence = "It was a fight that filled magazines and <mask> and gave professors case histories for business schools "
get_all_predictions(text_sentence)

It was a fight that filled magazines and <mask> and gave professors case histories for business schools 


{'bart': 'newspapers\nbooks\nnews\nblogs\nwebsites',
 'bert': 'newspapers\nbooks\npapers\nblogs\njournals',
 'electra': 'magazines\nbooks\nnewspapers\npapers\njournals',
 'roberta': 'newspapers\nbooks\ntextbooks\nwebsites\npapers',
 'xlm': 'newspaper\nbooks\narticles\npress\nnews',
 'xlnet': 'and\nbooks\ndid\ncolleges\nuniversities'}

In [ ]:
text_sentence = "To air fresh shows through June , `` we \'d have to eat the second <mask> of some episodes , \'\' he said , which would cause a financial hit . "
get_all_predictions(text_sentence)

To air fresh shows through June , `` we 'd have to eat the second <mask> of some episodes , '' he said , which would cause a financial hit . 


{'bart': 'half\nand\nor\nseason\nweek',
 'bert': 'half\nhalves\npart\nquarter\nseries',
 'electra': 'half\npart\nnight\nday\none',
 'roberta': 'half\npart\nquarter\nweek\nseason',
 'xlm': 'half\npart\nportion\nleg\nhour',
 'xlnet': 'half\nout\nfirst\nthird\nepisode'}

In [ ]:
text_sentence = "The lower court had enjoined the effect of its decision , pending the appeal , but the appeals court dissolved the <mask> . "
get_all_predictions(text_sentence)

The lower court had enjoined the effect of its decision , pending the appeal , but the appeals court dissolved the <mask> . 


{'bart': 'injunction\norder\nen\nstay\nlower',
 'bert': 'proceedings\ncourt\ncase\nsentence\nappeal',
 'electra': 'decision\nruling\nappeal\ncase\nproceedings',
 'roberta': 'injunction\norder\nstay\nappeal\nsuspension',
 'xlm': 'appeal\ncourt\ndecision\nargument\nevent',
 'xlnet': 'order\njudgment\nproceedings\ncase\nappeal'}

In [ ]:
text_sentence = "Mr. Hauke `` is looking for a much more costly repair that is nonstandard and would knock the <mask> beyond the the { $ 500} limit for a buy-back . \'\' "
get_all_predictions(text_sentence)

Mr. Hauke `` is looking for a much more costly repair that is nonstandard and would knock the <mask> beyond the the { $ 500} limit for a buy-back . '' 


{'bart': 'price\nvalue\nstock\ncompany\ncost',
 'bert': 'economy\ncompany\nindustry\nbusiness\ngovernment',
 'electra': 'price\nmoney\ncost\nprices\njob',
 'roberta': 'stock\ncompany\nshares\nprice\nstake',
 'xlm': 'repair\nprice\ndoor\nproperty\nowner',
 'xlnet': 'price\ncost\ncompany\nvalue\nrepair'}

In [ ]:
text_sentence = "They are irate about new capital-adequacy requirements that force securities firms to pump at least 20 <mask> more capital into reserves "
get_all_predictions(text_sentence)

They are irate about new capital-adequacy requirements that force securities firms to pump at least 20 <mask> more capital into reserves 


{'bart': 'percent\nper\ntimes\nbillion\npercentage',
 'bert': 'times\npercent\nyears\npounds\nfold',
 'electra': 'percent\ntimes\nor\nyears\nto',
 'roberta': 'percent\ntimes\nx\nX\npc',
 'xlm': 'times\npercent\nyears\nmillion\nbillion',
 'xlnet': 'more\npercent\n20\nbillion\nfurther'}

In [ ]:
text_sentence = "When the chairman of Santa Fe Southern Pacific Corp. flew into town a few months ago in the corporate jet , a small group of trainmen here knew they had struck a raw <mask> ."
get_all_predictions(text_sentence)

When the chairman of Santa Fe Southern Pacific Corp. flew into town a few months ago in the corporate jet , a small group of trainmen here knew they had struck a raw <mask> .


{'bart': 'nerve\ndeal\nchord\nnote\npolitical',
 'bert': 'nerve\nblow\nmatch\ntarget\ndeal',
 'electra': 'nerve\ndeal\nearthquake\nbullet\ndisaster',
 'roberta': 'deal\nnerve\npoint\nchord\nbargain',
 'xlm': 'water\nevent\nfire\npress\nshock',
 'xlnet': 'deal\nnerve\nR\none\npoint'}

In [ ]:
text_sentence = "Meanwhile , the discounters and specialty stores that have grabbed <mask> from Sears in recent years are n\'t likely to stand still while it tries to catch up ."
get_all_predictions(text_sentence)

Meanwhile , the discounters and specialty stores that have grabbed <mask> from Sears in recent years are n't likely to stand still while it tries to catch up .


{'bart': 'market\na\nmore\nthe\ncustomers',
 'bert': 'money\norders\nbooks\ninventory\naway',
 'electra': 'attention\nmoney\nprofits\naway\nrevenue',
 'roberta': 'ground\nshare\nbusiness\ncash\nshares',
 'xlm': 'customers\naway\nit\nmoney\nstock',
 'xlnet': 'up\ngrabbed\naway\nfrom\nmoney'}

In [ ]:
text_sentence = "In recent years , Drexel has poured far more <mask> into areas such as high-yield , `` junk \'\' bonds , merger and acquisition advice and mortgage-backed securities ."
get_all_predictions(text_sentence)

In recent years , Drexel has poured far more <mask> into areas such as high-yield , `` junk '' bonds , merger and acquisition advice and mortgage-backed securities .


{'bart': 'of\nmoney\nthan\ncapital\nresources',
 'bert': 'money\nresources\ncapital\nexpertise\neffort',
 'electra': 'money\ncash\nresources\ncapital\nenergy',
 'roberta': 'money\nresources\ncapital\ncash\nfunds',
 'xlm': 'money\ncapital\ninvestment\nenergy\nresources',
 'xlnet': 'resources\nmoney\nattention\nresearch\nexpertise'}

In [ ]:
text_sentence = "FIAT S.P.A . , the Italian auto maker , said it agreed to step up its <mask> with CIA. TELEFONICA NACIONAL DE ESPANA S.A. of Spain in the manufacturing of telecommunications equipment ."
get_all_predictions(text_sentence)

FIAT S.P.A . , the Italian auto maker , said it agreed to step up its <mask> with CIA. TELEFONICA NACIONAL DE ESPANA S.A. of Spain in the manufacturing of telecommunications equipment .


{'bart': 'cooperation\nefforts\nwork\ninvestment\njoint',
 'bert': 'collaboration\ncooperation\nrelationship\nwork\npartnership',
 'electra': 'partnership\ncooperation\ncollaboration\nrelationship\nrole',
 'roberta': 'cooperation\ncollaboration\npartnership\nwork\nactivities',
 'xlm': 'relationship\nassociation\npartnership\ntrade\nrelationships',
 'xlnet': 'cooperation\ncollaboration\npartnership\nrelations\nwork'}

In [ ]:
text_sentence = "But Mr. Sherlund said he thinks the company has taken steps to fix that <mask> ."
get_all_predictions(text_sentence)

But Mr. Sherlund said he thinks the company has taken steps to fix that <mask> .


{'bart': 'problem\nissue\nand\nsituation\nin',
 'bert': 'problem\nissue\nmess\nsituation\ncrisis',
 'electra': 'problem\nissue\nmess\nsituation\nscandal',
 'roberta': 'problem\nissue\nflaw\ndefect\nmistake',
 'xlm': 'issue\nproblem\nbug\nrepair\nincident',
 'xlnet': 'that\nproblem\na\ntoo\nissue'}